In [6]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

from langchain_community.embeddings import HuggingFaceEmbeddings

from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA


In [7]:
#Read pdfs from folder
loader=PyPDFDirectoryLoader("./us_census")

documents=loader.load()

text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)

final_documents=text_splitter.split_documents(documents)
final_documents[0]

Document(page_content='Health Insurance Coverage Status and Type \nby Geography: 2021 and 2022\nAmerican Community Survey Briefs\nACSBR-015Issued September 2023Douglas Conway and Breauna Branch\nINTRODUCTION\nDemographic shifts as well as economic and govern-\nment policy changes can affect people’s access to health coverage. For example, between 2021 and 2022, the labor market continued to improve, which may have affected private coverage in the United States \nduring that time.\n1 Public policy changes included \nthe renewal of the Public Health Emergency, which \nallowed Medicaid enrollees to remain covered under the Continuous Enrollment Provision.\n2 The American \nRescue Plan (ARP) enhanced Marketplace premium subsidies for those with incomes above 400 percent of the poverty level as well as for unemployed people.\n3', metadata={'source': 'us_census\\acsbr-015.pdf', 'page': 0})

In [8]:
len(final_documents)

316

In [11]:
huggingface_embeddings=HuggingFaceEmbeddings(
    model_name="BAAI/bge-small-en-v1.5",      #sentence-transformers/all-MiniLM-l6-v2
    model_kwargs={'device':'cpu'},
    encode_kwargs={'normalize_embeddings':True}
)

c:\Users\Arpith Paida\Documents\Projects\LANGCHAIN1\venv\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
c:\Users\Arpith Paida\Documents\Projects\LANGCHAIN1\venv\lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Arpith Paida\.cache\huggingface\hub\models--BAAI--bge-small-en-v1.5. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to act

In [18]:
import numpy as np
print(np.array(huggingface_embeddings.embed_query(final_documents[0].page_content)))
print(np.array(huggingface_embeddings.embed_query(final_documents[0].page_content)).shape)

[-7.81247392e-02 -1.76957231e-02 -2.55395211e-02  3.16485800e-02
  4.89566810e-02  5.73916398e-02 -3.41919921e-02  2.61430833e-02
 -9.73034725e-02 -2.28453204e-02  7.35054463e-02  5.15417606e-02
 -1.08876303e-02 -3.11451629e-02  3.21607059e-03  4.89183180e-02
 -1.29311206e-02 -1.42326951e-03 -3.19030881e-02  2.54794806e-02
 -3.13170739e-02  3.35001871e-02 -2.71926541e-02 -4.65958938e-02
  1.61344726e-02  1.42099531e-02 -7.43198395e-03  1.13744894e-02
 -5.09818569e-02 -1.41039521e-01  4.36503999e-03  2.54208185e-02
 -5.71403801e-02 -1.06509645e-02  2.70821210e-02  1.18388077e-02
 -2.27215644e-02  7.71624744e-02  4.27384116e-02  6.18330725e-02
 -3.68239954e-02  4.53975005e-03 -2.01893207e-02 -4.70246794e-03
 -2.41782814e-02 -5.31231235e-05 -3.19753475e-02 -4.51396545e-03
  3.41199874e-03 -5.89688681e-02  2.63294503e-02 -8.91797245e-03
  5.06487750e-02  6.75783083e-02  6.58879504e-02 -4.71076258e-02
  2.60821730e-02 -2.50375960e-02 -4.24258895e-02  3.94773334e-02
  4.30341288e-02 -3.05915

In [22]:
vectorstore=Chroma.from_documents(final_documents[:120],huggingface_embeddings)

In [23]:
query="what is health insurance coverage"
relevant_documents=vetorstore.similarity_search(query)
print(relevant_documents[0].page_content)

NameError: name 'vetorstore' is not defined

In [ ]:
#get 3 results
retriever=vectorstore.as_retriever(search_type="similarity",search_kwargs={"k":3})
print(retriever)

In [ ]:

import os
os.environ['HUGGINGFACEHUB_API_TOKEN']=""

The Hugging Face Hub is an platform with over 350k models, 75k datasets, and 150k demo apps (Spaces), all open source and publicly available, in an online platform where people can easily collaborate and build ML together.

In [5]:
from langchain_community.llms import HuggingFaceHub

hf=HuggingFaceHub(
    repo_id="mistralai/Mistral-7B-v0.1",
    model_kwargs={"temperature":0.1,"max_length":500}

)
query="What is the health insurance coverage?"
hf.invoke(query)

316

In [ ]:
#Hugging Face models can be run locally through the HuggingFacePipeline class.
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline

hf = HuggingFacePipeline.from_model_id(
    model_id="mistralai/Mistral-7B-v0.1",
    task="text-generation",
    pipeline_kwargs={"temperature": 0, "max_new_tokens": 300}
)

llm = hf 
llm.invoke(query)

In [ ]:
prompt_template="""
Use the following piece of context to answer the question asked.
Please try to provide the answer only based on the context

{context}
Question:{question}

Helpful Answers:
 """

In [ ]:
retrievalQA=RetrievalQA.from_chain_type(
    llm=hf,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt":prompt}
)

In [ ]:
query="""DIFFERENCES IN THE
UNINSURED RATE BY STATE
IN 2022"""

In [ ]:

# Call the QA chain with our query.
result = retrievalQA.invoke({"query": query})
print(result['result'])